In [9]:
import numpy as np
from collections import defaultdict
from load_data import load_process_essays

import logging
import datetime
import pandas as pd

from Settings import Settings
settings = Settings()

Results Dir: /Users/simon.hughes/Google Drive/Phd/Results/
Data Dir:    /Users/simon.hughes/Google Drive/Phd/Data/
Root Dir:    /Users/simon.hughes/GitHub/NlpResearch/
Public Data: /Users/simon.hughes/GitHub/NlpResearch/Data/PublicDatasets/


## Load and Pre-Process Essays

In [13]:
DATASET = "CoralBleaching"
# DATASET = "SkinCancer"

In [27]:
folder = settings.data_directory + DATASET + "/Thesis_Dataset/Training"
br_train_essays = load_bratt_essays(folder)

901 files found
901 essays processed


In [28]:
folder = settings.data_directory + DATASET + "/Thesis_Dataset/Test"
br_test_essays = load_bratt_essays(folder)

226 files found
226 essays processed


In [29]:
br_essays = br_train_essays + br_test_essays
len(br_essays)

1127

In [16]:
import pickle
from CrossValidation import cross_validation
from BrattEssay import load_bratt_essays
from IterableFP import flatten
from Settings import Settings
from window_based_tagger_config import get_config

CV_FOLDS = 5
DEV_SPLIT = 0.1

settings = Settings()
# data_set = "SkinCancer"
data_set = "CoralBleaching"

root_folder = settings.data_directory + data_set + "/Thesis_Dataset/"
training_folder = root_folder + "Training" + "/"
test_folder = root_folder + "Test" + "/"
training_pickled = settings.data_directory + data_set + "/Thesis_Dataset/training.pl"
models_folder = root_folder + "Models/Bi-LSTM/"

config = get_config(training_folder)

Results Dir: /Users/simon.hughes/Google Drive/Phd/Results/
Data Dir:    /Users/simon.hughes/Google Drive/Phd/Data/
Root Dir:    /Users/simon.hughes/GitHub/NlpResearch/
Public Data: /Users/simon.hughes/GitHub/NlpResearch/Data/PublicDatasets/


In [17]:
%%time
config = get_config(training_folder)
train_essays = load_process_essays(**config)

901 files found
901 essays processed
CPU times: user 43.3 s, sys: 203 ms, total: 43.5 s
Wall time: 43.5 s


In [18]:
%%time
config = get_config(test_folder)
test_essays = load_process_essays(**config)

226 files found
226 essays processed
CPU times: user 14.8 s, sys: 95.2 ms, total: 14.9 s
Wall time: 15 s


## Prepare Tags

In [46]:
import BrattEssay

In [48]:
def get_essay_stats(essays, lbl):
    tag_freq = defaultdict(int)
    unique_words = set()
    num_wds = 0
    num_sents = 0
    
    is_bratt_essay = (type(essays[0]) == BrattEssay.Essay)
    if is_bratt_essay:
        get_sents = lambda e: e.tagged_sentences
    else:
        get_sents = lambda e: e.sentences
    
    for essay in essays:
        for sentence in get_sents(essay):
            num_sents += 1
            for word, tags in sentence:
                num_wds += 1
                unique_words.add(word)
    return {
        "dataset": lbl,
        "num_essays": len(essays),
        "num_sents": num_sents,
        "num_words": num_wds,
        "vocab": len(unique_words)        
    }

In [49]:
all_essays = train_essays + test_essays

a = get_essay_stats(train_essays, "train")
b = get_essay_stats(test_essays,  "test")
c = get_essay_stats(all_essays,    "all")

pd.DataFrame([a,b,c])

,dataset,num_essays,num_sents,num_words,vocab
0,train,901,8280,136948,1641
1,test,226,1918,30699,927
2,all,1127,10198,167647,1677


In [31]:
len(all_essays), len(br_essays)

(1127, 1127)

In [50]:
e = br_essays[0]
# e.tagged_sentences

In [51]:
pd.DataFrame([get_essay_stats(br_essays, "Bratt Essays")])

,dataset,num_essays,num_sents,num_words,vocab
0,Bratt Essays,1127,10198,167656,4770


In [7]:
# CB

# dataset	    num_essys	num_sents	num_words	vocab
# 0	train	    901     	8280    	136948  	1641
# 1	test	    226     	1918    	30699   	927
# 2	all     	1127    	10198   	167647  	1677

In [8]:
# SC

# dataset	num_essays	num_sents	num_words	vocab
# 0	train	870     	8573    	145471  	1582
# 1	test	218     	2097    	35402   	899
# 2	all 	1088    	10670   	180873  	1623